In [2]:
import pandas as pd

import sklearn
from sklearn import svm
from sklearn.model_selection import train_test_split

from feature_engineering import feature_engineering
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from feature_builder import process_dataset

[nltk_data] Downloading package stopwords to /home/andres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train_dataset = pd.read_csv('train.csv')

In [4]:
test_dataset = pd.read_csv('test.csv')

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x_tfidf = v.fit_transform(train_dataset['text'])


In [8]:
y = train_dataset.loc[:,'target']

In [8]:
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(x_tfidf, y, test_size = .33, random_state = 17)

In [9]:
clf_tfidf = svm.SVC()
clf_tfidf.fit(x_train_tfidf, y_train_tfidf)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [12]:
y_pred_tfidf = clf_tfidf.predict(x_test_tfidf)
accuracy_score(y_test_tfidf, y_pred_tfidf)

0.8010346199761241

Ahora voy a probar con nuestro generador de features que es un conjunto de embeddings con manualidades.

In [32]:
x_processed = process_dataset(train_dataset)

In [33]:
x_train_processed, x_test_processed, y_train_processed, y_test_processed = train_test_split(x_processed, y, test_size = .33, random_state = 17)

In [34]:
clf_processed = svm.SVC()
clf_processed.fit(x_train_processed, y_train_processed)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [35]:
y_pred_processed = clf_processed.predict(x_test_processed)
accuracy_score(y_test_processed, y_pred_processed)

0.6327099084759252

Como no son muy buenos resultados voy a probar solo con los embeddings (las primeras 300 columnas)

In [36]:
x_embedd = x_processed.iloc[:,:299]

Me quede solo con los embeddings ya que el resto deteriora el algoritmo por alguna razón.

In [37]:
x_train_embedd, x_test_embedd, y_train_embedd, y_test_embedd = train_test_split(x_embedd, y, test_size = .33, random_state = 17)

In [38]:
clf_embeddings = svm.SVC()
clf_embeddings.fit(x_train_embedd, y_train_embedd)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
y_pred_embedd = clf_embeddings.predict(x_test_embedd)
accuracy_score(y_test_embedd, y_pred_embedd)

0.8241146040588938

Acá obtuve un resultado mucho mejor, por ende lo voy a entrenar de esta manera para todo el dataset y exportarlo a csv.

In [12]:
x_train = (process_dataset(train_dataset)).iloc[:,:299]

In [13]:
x_test = (process_dataset(test_dataset)).iloc[:,:299]

In [28]:
clf_embeddings_final = svm.SVC()
clf_embeddings_final.fit(x_train, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [29]:
y_pred_embedd = clf_embeddings_final.predict(x_test)

In [30]:
ids = test_dataset['id']
final_df = pd.DataFrame({'target': [x for x in y_pred_embedd]}, index=ids)

In [31]:
final_df.to_csv('SVM-algo3.csv')